In [2]:
# For Full Project >> github.com/0PeterAdel/Brazilian-ECommerce


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_products_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_geolocation_dataset.csv
/kaggle/input/brazilian-ecommerce/product_category_name_translation.csv
/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv


**Load the All DataSet**

In [16]:
root_path = "/kaggle/input/brazilian-ecommerce/"
output_path = "/kaggle/working/"

customers = pd.read_csv(root_path + 'olist_customers_dataset.csv')
sellers = pd.read_csv(root_path + 'olist_sellers_dataset.csv')
order_reviews = pd.read_csv(root_path + 'olist_order_reviews_dataset.csv')
order_items = pd.read_csv(root_path + 'olist_order_items_dataset.csv')
products = pd.read_csv(root_path + 'olist_products_dataset.csv')
geolocation = pd.read_csv(root_path + 'olist_geolocation_dataset.csv')
categories = pd.read_csv(root_path + 'product_category_name_translation.csv')
orders = pd.read_csv(root_path + 'olist_orders_dataset.csv')
order_payments = pd.read_csv(root_path + 'olist_order_payments_dataset.csv')


---
---

## 1.  olist_customers_dataset

In [18]:
# 1. Inspect the data

print("Initial Customers Dataset Info:")
print(customers.info())
print("\nMissing Values:")
print(customers.isnull().sum())

Initial Customers Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  object
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
 5   is_valid_state            99441 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 4.6+ MB
None

Missing Values:
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
is_valid_state              0
dtype: int64


In [17]:
# 2. Drop rows with missing critical IDs
customers = customers.dropna(subset=['customer_id', 'customer_unique_id'])

# 3. Fill missing geographic data with 'Unknown'
customers['customer_zip_code_prefix'] = customers['customer_zip_code_prefix'].fillna('Unknown')
customers['customer_city'] = customers['customer_city'].fillna('Unknown')
customers['customer_state'] = customers['customer_state'].fillna('Unknown')

# 4. Convert to consistent data types
customers['customer_id'] = customers['customer_id'].astype(str)
customers['customer_unique_id'] = customers['customer_unique_id'].astype(str)
customers['customer_zip_code_prefix'] = customers['customer_zip_code_prefix'].astype(str)

# 5. Standardize text
customers['customer_city'] = customers['customer_city'].str.strip().str.title()
customers['customer_state'] = customers['customer_state'].str.strip().str.upper()

# 6. Remove duplicate customer_ids
customers = customers.drop_duplicates(subset=['customer_id'], keep='first')

# 7. Creative addition: Validate Brazilian state codes
valid_states = {'AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 
                'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 
                'RR', 'SC', 'SP', 'SE', 'TO'}
customers['is_valid_state'] = customers['customer_state'].isin(valid_states).astype(int)

# 8. Save the cleaned file
customers.to_csv(output_path + 'cleaned_olist_customers_dataset.csv', index=False)
print("Saved cleaned customers dataset as '/Data_Cleaned/cleaned_olist_customers_dataset.csv'")

Saved cleaned customers dataset as '/Data_Cleaned/cleaned_olist_customers_dataset.csv'


---
---
## 2.  olist_sellers_dataset

In [19]:
# 1. Inspect the data

print("Initial Sellers Dataset Info:")
print(sellers.info())
print("\nMissing Values:")
print(sellers.isnull().sum())
print("\nDuplicate Seller IDs:")
print(sellers['seller_id'].duplicated().sum())

Initial Sellers Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB
None

Missing Values:
seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

Duplicate Seller IDs:
0


In [20]:
# 2. Handle missing values
# Check for missing values in critical columns
if sellers['seller_id'].isnull().sum() > 0:
    sellers = sellers.dropna(subset=['seller_id'])  # Drop rows with missing seller_id
    print("Dropped rows with missing seller_id")

# For zip code, city, and state, fill missing values with 'Unknown' to preserve data
sellers['seller_zip_code_prefix'] = sellers['seller_zip_code_prefix'].fillna('Unknown')
sellers['seller_city'] = sellers['seller_city'].fillna('Unknown')
sellers['seller_state'] = sellers['seller_state'].fillna('Unknown')

# 3. Ensure data type consistency
# seller_id should be string
sellers['seller_id'] = sellers['seller_id'].astype(str)
# seller_zip_code_prefix might be numeric or string; convert to string for consistency
sellers['seller_zip_code_prefix'] = sellers['seller_zip_code_prefix'].astype(str)
# seller_city and seller_state should be strings
sellers['seller_city'] = sellers['seller_city'].astype(str)
sellers['seller_state'] = sellers['seller_state'].astype(str)

# 4. Standardize text data
# Remove leading/trailing whitespace and convert cities/states to title case
sellers['seller_city'] = sellers['seller_city'].str.strip().str.title()
sellers['seller_state'] = sellers['seller_state'].str.strip().str.upper()

# 5. Check for duplicates
# Remove duplicate seller_id entries, keeping the first occurrence
sellers = sellers.drop_duplicates(subset=['seller_id'], keep='first')
print(f"Removed {sellers['seller_id'].duplicated().sum()} duplicate seller IDs")

# 6. Validate state codes (Creative Touch)
# Brazilian states have 2-letter codes; flag any anomalies
valid_states = {'AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 
                'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 
                'RR', 'SC', 'SP', 'SE', 'TO'}
sellers['is_valid_state'] = sellers['seller_state'].isin(valid_states).astype(int)
invalid_states = sellers[~sellers['seller_state'].isin(valid_states)]['seller_state'].unique()
if len(invalid_states) > 0:
    print(f"Invalid state codes detected: {invalid_states}")

# 7. Creative Addition - Derive region from state
# Mapping Brazilian states to regions for geographic analysis
state_to_region = {
    'AC': 'North', 'AM': 'North', 'AP': 'North', 'PA': 'North', 'RO': 'North', 'RR': 'North', 'TO': 'North',
    'AL': 'Northeast', 'BA': 'Northeast', 'CE': 'Northeast', 'MA': 'Northeast', 'PB': 'Northeast', 
    'PE': 'Northeast', 'PI': 'Northeast', 'RN': 'Northeast', 'SE': 'Northeast',
    'DF': 'Central-West', 'GO': 'Central-West', 'MT': 'Central-West', 'MS': 'Central-West',
    'ES': 'Southeast', 'MG': 'Southeast', 'RJ': 'Southeast', 'SP': 'Southeast',
    'PR': 'South', 'RS': 'South', 'SC': 'South'
}
sellers['seller_region'] = sellers['seller_state'].map(state_to_region).fillna('Unknown')
print("Added seller_region column for geographic insights")


# 8. Save the cleaned dataset
sellers.to_csv(output_path + 'cleaned_sellers_dataset.csv', index=False)
print("Sellers dataset cleaned and saved as 'cleaned_sellers_dataset.csv'")

Removed 0 duplicate seller IDs
Added seller_region column for geographic insights
Sellers dataset cleaned and saved as 'cleaned_sellers_dataset.csv'


---
---
## 3. olist_order_reviews_dataset

In [22]:
# 1. Inspect the data

print("Initial Order Reviews Dataset Info:")
print(order_reviews.info())
print("\nMissing Values:")
print(order_reviews.isnull().sum())

Initial Order Reviews Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB
None

Missing Values:
review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64


In [24]:
# 2. Drop rows with missing critical IDs
order_reviews = order_reviews.dropna(subset=['review_id', 'order_id'])

# 3. Fill missing comments
order_reviews['review_comment_title'] = order_reviews['review_comment_title'].fillna('No Comment')
order_reviews['review_comment_message'] = order_reviews['review_comment_message'].fillna('No Comment')

# 4. Convert data types
order_reviews['review_id'] = order_reviews['review_id'].astype(str)
order_reviews['order_id'] = order_reviews['order_id'].astype(str)
order_reviews['review_score'] = order_reviews['review_score'].astype(int)
order_reviews['review_creation_date'] = pd.to_datetime(order_reviews['review_creation_date'], errors='coerce')
order_reviews['review_answer_timestamp'] = pd.to_datetime(order_reviews['review_answer_timestamp'], errors='coerce')

# 5. Validate review scores
order_reviews = order_reviews[order_reviews['review_score'].between(1, 5)]

# 6. Creative addition: Calculate response time
order_reviews['response_time_days'] = (order_reviews['review_answer_timestamp'] - 
                                       order_reviews['review_creation_date']).dt.days

# 7. Save the cleaned file
order_reviews.to_csv(output_path + 'cleaned_olist_order_reviews_dataset.csv', index=False)
print("Saved cleaned order reviews dataset as 'cleaned_olist_order_reviews_dataset.csv'")

Saved cleaned order reviews dataset as 'cleaned_olist_order_reviews_dataset.csv'


---
---
## 4. olist_order_items_dataset

In [25]:
# 1. Inspect the data

print("Initial Order Items Dataset Info:")
print(order_items.info())
print("\nMissing Values:")
print(order_items.isnull().sum())

Initial Order Items Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB
None

Missing Values:
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64


In [26]:
# 2. Drop rows with missing critical IDs
order_items = order_items.dropna(subset=['order_id', 'product_id', 'seller_id'])

# 3. Fill missing shipping_limit_date
order_items['shipping_limit_date'] = order_items['shipping_limit_date'].fillna('2099-12-31')

# 4. Convert data types
order_items['order_id'] = order_items['order_id'].astype(str)
order_items['product_id'] = order_items['product_id'].astype(str)
order_items['seller_id'] = order_items['seller_id'].astype(str)
order_items['shipping_limit_date'] = pd.to_datetime(order_items['shipping_limit_date'], errors='coerce')
order_items['price'] = order_items['price'].astype(float)
order_items['freight_value'] = order_items['freight_value'].astype(float)

# 5. Ensure non-negative values
order_items['price'] = order_items['price'].clip(lower=0)
order_items['freight_value'] = order_items['freight_value'].clip(lower=0)

# 6. Creative addition: Calculate total cost
order_items['total_cost'] = order_items['price'] + order_items['freight_value']

# 7. Save the cleaned file
order_items.to_csv(output_path + 'cleaned_olist_order_items_dataset.csv', index=False)
print("Saved cleaned order items dataset as 'cleaned_olist_order_items_dataset.csv'")

Saved cleaned order items dataset as 'cleaned_olist_order_items_dataset.csv'


---
---
## 5. olist_products_dataset

In [27]:
# 1. Inspect the data

print("Initial Products Dataset Info:")
print(products.info())
print("\nMissing Values:")
print(products.isnull().sum())
print("\nDuplicate Product IDs:")
print(products['product_id'].duplicated().sum())

Initial Products Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB
None

Missing Values:
product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty      

In [28]:
# 2. Handle missing values
# Drop rows with missing product_id
products = products.dropna(subset=['product_id'])
print("Dropped rows with missing product_id")

# Fill missing product_category_name with 'Unknown'
products['product_category_name'] = products['product_category_name'].fillna('Unknown')

# Numerical columns: impute with median to maintain distribution
numeric_cols = ['product_name_lenght', 'product_description_lenght', 'product_photos_qty',
                'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']
for col in numeric_cols:
    products[col] = products[col].fillna(products[col].median())
    print(f"Imputed missing values in {col} with median")
    
# 3. Ensure data type consistency
products['product_id'] = products['product_id'].astype(str)
products['product_category_name'] = products['product_category_name'].astype(str)
for col in numeric_cols:
    products[col] = products[col].astype(float)  # Use float to accommodate potential decimals

# 4. Validate numerical data
# Check for negative or unrealistic values
for col in numeric_cols:
    if (products[col] < 0).sum() > 0:
        products.loc[products[col] < 0, col] = products[col].median()
        print(f"Replaced negative values in {col} with median")

    # Cap extreme outliers (e.g., > 99th percentile)
    upper_limit = products[col].quantile(0.99)
    products[col] = products[col].clip(lower=0, upper=upper_limit)  # Fixed syntax
    print(f"Capped {col} at 99th percentile: {upper_limit}")

# 5. Check for duplicates
products = products.drop_duplicates(subset=['product_id'], keep='first')
print(f"Removed {products['product_id'].duplicated().sum()} duplicate product IDs")

# 6. Creative Addition - Calculate product volume
products['product_volume_cm3'] = (products['product_length_cm'] * 
                                  products['product_height_cm'] * 
                                  products['product_width_cm'])
print("Added product_volume_cm3 for size-related analysis")

# 7. Creative Addition - Flag heavy products
products['is_heavy'] = (products['product_weight_g'] > products['product_weight_g'].quantile(0.75)).astype(int)
print("Added is_heavy flag for logistics insights")

# 8. Save the cleaned dataset
products.to_csv(output_path + 'cleaned_products_dataset.csv', index=False)
print("Products dataset cleaned and saved as 'cleaned_products_dataset.csv'")

Dropped rows with missing product_id
Imputed missing values in product_name_lenght with median
Imputed missing values in product_description_lenght with median
Imputed missing values in product_photos_qty with median
Imputed missing values in product_weight_g with median
Imputed missing values in product_length_cm with median
Imputed missing values in product_height_cm with median
Imputed missing values in product_width_cm with median
Capped product_name_lenght at 99th percentile: 63.0
Capped product_description_lenght at 99th percentile: 3274.5
Capped product_photos_qty at 99th percentile: 8.0
Capped product_weight_g at 99th percentile: 22537.5
Capped product_length_cm at 99th percentile: 100.0
Capped product_height_cm at 99th percentile: 69.0
Capped product_width_cm at 99th percentile: 63.0
Removed 0 duplicate product IDs
Added product_volume_cm3 for size-related analysis
Added is_heavy flag for logistics insights
Products dataset cleaned and saved as 'cleaned_products_dataset.csv'


---
---
## 6. olist_geolocation_dataset

In [30]:
# 1. Inspect the data

print("Initial Geolocation Dataset Info:")
print(geolocation.info())
print("\nMissing Values:")
print(geolocation.isnull().sum())

Initial Geolocation Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB
None

Missing Values:
geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64


In [31]:
# 2. Fill missing city and state
geolocation['geolocation_city'] = geolocation['geolocation_city'].fillna('Unknown')
geolocation['geolocation_state'] = geolocation['geolocation_state'].fillna('Unknown')

# 3. Convert data types
geolocation['geolocation_zip_code_prefix'] = geolocation['geolocation_zip_code_prefix'].astype(str)
geolocation['geolocation_lat'] = geolocation['geolocation_lat'].astype(float)
geolocation['geolocation_lng'] = geolocation['geolocation_lng'].astype(float)

# 4. Standardize text
geolocation['geolocation_city'] = geolocation['geolocation_city'].str.strip().str.title()
geolocation['geolocation_state'] = geolocation['geolocation_state'].str.strip().str.upper()

# 5. Remove duplicates
geolocation = geolocation.drop_duplicates(subset=['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng'], keep='first')

# 6. Creative addition: Validate lat/lng for Brazil (lat: -33.75 to 5.27, lng: -73.99 to -32.39)
geolocation['is_valid_coords'] = (geolocation['geolocation_lat'].between(-33.75, 5.27) & 
                                  geolocation['geolocation_lng'].between(-73.99, -32.39)).astype(int)

# 7. Save the cleaned file
geolocation.to_csv(output_path + 'cleaned_olist_geolocation_dataset.csv', index=False)
print("Saved cleaned geolocation dataset as 'cleaned_olist_geolocation_dataset.csv'")

Saved cleaned geolocation dataset as 'cleaned_olist_geolocation_dataset.csv'


---
---
## 7. product_category_name_translation

In [32]:
# 1. Inspect the data

print("Initial Categories Dataset Info:")
print(categories.info())
print("\nMissing Values:")
print(categories.isnull().sum())
print("\nDuplicate Categories:")
print(categories['product_category_name'].duplicated().sum())

Initial Categories Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB
None

Missing Values:
product_category_name            0
product_category_name_english    0
dtype: int64

Duplicate Categories:
0


In [34]:
# 2. Handle missing values
# Drop rows with missing category names (either Portuguese or English)
categories = categories.dropna(subset=['product_category_name', 'product_category_name_english'])
print(f"Dropped {categories.isnull().sum().sum()} rows with missing category names")

# 3. Ensure data type consistency
categories['product_category_name'] = categories['product_category_name'].astype(str)
categories['product_category_name_english'] = categories['product_category_name_english'].astype(str)

# 4. Standardize text data
# Remove whitespace and standardize formatting
categories['product_category_name'] = categories['product_category_name'].str.strip().str.lower()
categories['product_category_name_english'] = categories['product_category_name_english'].str.strip().str.lower().str.replace(' ', '_')

# 5. Check for duplicates
# Remove duplicate Portuguese category names, keeping the first translation
categories = categories.drop_duplicates(subset=['product_category_name'], keep='first')
print(f"Removed {categories['product_category_name'].duplicated().sum()} duplicate category names")

# 6. Creative Addition - Add category grouping
# Group categories into broader supercategories for advanced analysis
supercategory_mapping = {
    'health_beauty': 'Personal Care', 'perfumery': 'Personal Care', 'baby': 'Personal Care',
    'computers_accessories': 'Electronics', 'telephony': 'Electronics', 'fixed_telephony': 'Electronics',
    'electronics': 'Electronics', 'home_appliances': 'Electronics', 'home_appliances_2': 'Electronics',
    'auto': 'Automotive', 'construction_tools_construction': 'Home & Garden', 'garden_tools': 'Home & Garden',
    'bed_bath_table': 'Home & Garden', 'furniture_decor': 'Home & Garden', 'housewares': 'Home & Garden',
    'sports_leisure': 'Sports & Leisure', 'fashion_bags_accessories': 'Fashion', 'fashion_shoes': 'Fashion',
    'fashion_male_clothing': 'Fashion', 'fashion_underwear_beach': 'Fashion', 'fashion_sport': 'Fashion',
    'fashion_female_clothing': 'Fashion', 'fashion_childrens_clothes': 'Fashion',
    'toys': 'Entertainment', 'consoles_games': 'Entertainment', 'musical_instruments': 'Entertainment',
    'food_drink': 'Food & Beverage', 'food': 'Food & Beverage', 'drinks': 'Food & Beverage',
    # Add more mappings as needed
}
categories['supercategory'] = categories['product_category_name_english'].map(supercategory_mapping).fillna('Other')
print("Added supercategory column for broader category analysis")

# 7. Save the cleaned dataset
categories.to_csv(output_path + 'cleaned_product_category_name_translation.csv', index=False)
print("Categories dataset cleaned and saved as 'cleaned_product_category_name_translation.csv'")

Dropped 0 rows with missing category names
Removed 0 duplicate category names
Added supercategory column for broader category analysis
Categories dataset cleaned and saved as 'cleaned_product_category_name_translation.csv'


---
---
## 8. olist_orders_dataset

In [35]:
# 1. Inspect the data

print("Initial Orders Dataset Info:")
print(orders.info())
print("\nMissing Values:")
print(orders.isnull().sum())

Initial Orders Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB
None

Missing Values:
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carri

In [36]:
# 2. Drop rows with missing order_id
orders = orders.dropna(subset=['order_id'])

# 3. Fill missing timestamps
timestamp_cols = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date',
                  'order_delivered_customer_date', 'order_estimated_delivery_date']
for col in timestamp_cols:
    orders[col] = orders[col].fillna('2099-12-31')

# 4. Convert data types
orders['order_id'] = orders['order_id'].astype(str)
orders['customer_id'] = orders['customer_id'].astype(str)
orders['order_status'] = orders['order_status'].astype(str)
for col in timestamp_cols:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

# 5. Creative addition: Calculate delivery time
orders['delivery_time_days'] = (orders['order_delivered_customer_date'] - 
                                orders['order_purchase_timestamp']).dt.days

# 6. Save the cleaned file
orders.to_csv(output_path + 'cleaned_olist_orders_dataset.csv', index=False)
print("Saved cleaned orders dataset as 'cleaned_olist_orders_dataset.csv'")

Saved cleaned orders dataset as 'cleaned_olist_orders_dataset.csv'


---
---
## 9. olist_order_payments_dataset

In [37]:
# 1. Inspect the data

print("Initial Order Payments Dataset Info:")
print(order_payments.info())
print("\nMissing Values:")
print(order_payments.isnull().sum())

Initial Order Payments Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB
None

Missing Values:
order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64


In [38]:
# 2. Drop rows with missing order_id
order_payments = order_payments.dropna(subset=['order_id'])

# 3. Fill missing payment_type
order_payments['payment_type'] = order_payments['payment_type'].fillna('Unknown')

# 4. Convert data types
order_payments['order_id'] = order_payments['order_id'].astype(str)
order_payments['payment_type'] = order_payments['payment_type'].astype(str)
order_payments['payment_value'] = order_payments['payment_value'].astype(float)

# 5. Ensure non-negative payment values
order_payments['payment_value'] = order_payments['payment_value'].clip(lower=0)

# 6. Creative addition: Categorize payments
order_payments['payment_category'] = order_payments['payment_type'].apply(
    lambda x: 'Card' if 'card' in x.lower() else 'Other'
)

# 7. Save the cleaned file
order_payments.to_csv(output_path + 'cleaned_olist_order_payments_dataset.csv', index=False)
print("Saved cleaned order payments dataset as 'cleaned_olist_order_payments_dataset.csv'")

Saved cleaned order payments dataset as 'cleaned_olist_order_payments_dataset.csv'


---
---
## 1. Merging_Datasets_Cleaned

In [39]:
# 1. Load all cleaned datasets

orders = pd.read_csv(output_path + 'cleaned_olist_orders_dataset.csv')
order_items = pd.read_csv(output_path + 'cleaned_olist_order_items_dataset.csv')
products = pd.read_csv(output_path + 'cleaned_products_dataset.csv')
category_translation = pd.read_csv(output_path + 'cleaned_product_category_name_translation.csv')
sellers = pd.read_csv(output_path + 'cleaned_sellers_dataset.csv')
customers = pd.read_csv(output_path + 'cleaned_olist_customers_dataset.csv')
order_payments = pd.read_csv(output_path + 'cleaned_olist_order_payments_dataset.csv')
order_reviews = pd.read_csv(output_path + 'cleaned_olist_order_reviews_dataset.csv')

In [40]:
# 2. Merge step-by-step using left joins to preserve all orders
merged_data = orders.merge(order_items, on='order_id', how='left') \
                    .merge(products, on='product_id', how='left') \
                    .merge(category_translation, on='product_category_name', how='left') \
                    .merge(sellers, on='seller_id', how='left') \
                    .merge(customers, on='customer_id', how='left') \
                    .merge(order_payments, on='order_id', how='left') \
                    .merge(order_reviews, on='order_id', how='left')

In [41]:
# 3. Save the merged dataset
merged_data.to_csv(output_path + 'merged_olist_dataset.csv', index=False)
print("All cleaned datasets merged and saved as 'kaggle/working/merged_olist_dataset.csv'")


All cleaned datasets merged and saved as 'kaggle/working/merged_olist_dataset.csv'
